# Сбор данных вакансий Data Science / Data Analytics
 
## План:
### Определение поисковых запросов
### Сбор данных через API HH.ru
### Сохранение сырых данных
### Предварительный осмотр

### 1. Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import os
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

print('Библиотеки успешно загуржены')

Библиотеки успешно загуржены


### 2. Определяем параметры поиска

In [9]:
# Ключевые слова для поиска
SEARCH_KEYWORDS = [
    'data scientist',
    'data analyst',
    'аналитик данных',
    'дата саентист',
    'ML engineer',
    'machine learning'
]

# Регионы (коды с HH.ru)
REGIONS = {
    'Москва': '1',
    'Санкт-Петербург': '2',
    'Новосибирск': '4',
    'Екатеринбург': '3',
    'Казань': '88',
    'Россия': '113'  # Вся Россия
}

# Параметры API
BASE_URL = 'https://api.hh.ru/vacancies'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

In [8]:
REGIONS.get('Москва','None')

'1'

### 3. Функция для получения вакансий через API

In [11]:
def get_vacancies_from_hh(search_text='data scientist', area='113', per_page=100, days=30):
    """
    Получает вакансии с HH.ru API
    
    Parameters:
    -----------
    search_text : str
        Текст для поиска
    area : str
        Код региона (113 = Россия)
    per_page : int
        Количество вакансий на странице (max 100)
    days : int
        За сколько дней искать вакансии
    
    Returns:
    --------
    list
        Список словарей с вакансиями
    """
    vacancies = []
    
    # Параметры запроса
    params = {
        'text': search_text,
        'area': area,
        'per_page': per_page,
        'date_from': f'{(datetime.now() - pd.Timedelta(days=days)).strftime("%Y-%m-%d")}',
        'page': 0
    }
    
    try:
        # Получаем первую страницу для информации о количестве страниц
        response = requests.get(BASE_URL, params=params, headers=HEADERS)
        response.raise_for_status()
        data = response.json()
        
        total_pages = data.get('pages', 1)
        found = data.get('found', 0)
        
        print(f"По запросу '{search_text}' найдено {found} вакансий. Страниц: {total_pages}")
        
        # Ограничим количество страниц для теста
        pages_to_fetch = min(total_pages, 5)  # Берем первые 5 страниц для начала
        
        # Собираем вакансии со всех страниц
        for page in tqdm(range(pages_to_fetch), desc=f"Сбор '{search_text[:20]}...'"):
            params['page'] = page
            response = requests.get(BASE_URL, params=params, headers=HEADERS)
            response.raise_for_status()
            page_data = response.json()
            
            # Добавляем вакансии текущей страницы
            vacancies.extend(page_data.get('items', []))
            
            # Пауза чтобы не нагружать API
            time.sleep(0.1)
            
    except Exception as e:
        print(f"Ошибка при запросе: {e}")
    
    return vacancies

### 4. Собираем данные по всем ключевым словам

In [12]:
# Для начала соберем данные только по России и 2 ключевым словам (чтобы не перегружать API)
all_vacancies = []

# Тестовый сбор (минимум для начала)
test_keywords = ['data scientist', 'аналитик данных']
test_region = '113'  # Россия

for keyword in test_keywords:
    print(f"\nСбор вакансий по запросу: '{keyword}'")
    vacs = get_vacancies_from_hh(
        search_text=keyword,
        area=test_region,
        per_page=50,  # 50 вакансий на страницу
        days=7  # Вакансии за последние 7 дней
    )
    all_vacancies.extend(vacs)
    print(f"Собрано: {len(vacs)} вакансий")
    time.sleep(1)  # Пауза между запросами

print(f"\nВсего собрано вакансий: {len(all_vacancies)}")


Сбор вакансий по запросу: 'data scientist'
По запросу 'data scientist' найдено 118 вакансий. Страниц: 3


Сбор 'data scientist...':   0%|          | 0/3 [00:00<?, ?it/s]

Собрано: 118 вакансий

Сбор вакансий по запросу: 'аналитик данных'
По запросу 'аналитик данных' найдено 5101 вакансий. Страниц: 40


Сбор 'аналитик данных...':   0%|          | 0/5 [00:00<?, ?it/s]

Собрано: 250 вакансий

Всего собрано вакансий: 368


### 5. Сохраняем сырые данные

In [13]:
# Сохраняем в JSON
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
raw_data_path = f'../data/raw/vacancies_raw_{timestamp}.json'

with open(raw_data_path, 'w', encoding='utf-8') as f:
    json.dump(all_vacancies, f, ensure_ascii=False, indent=2)

print(f"Данные сохранены в: {raw_data_path}")
print(f"Количество вакансий: {len(all_vacancies)}")


Данные сохранены в: ../data/raw/vacancies_raw_20251216_192026.json
Количество вакансий: 368


### 6. Предварительный просмотр данных

In [14]:
# Создаем DataFrame для предварительного анализа
df_raw = pd.DataFrame(all_vacancies)

print("Размерность данных:", df_raw.shape)
print("\nКолонки в данных:")
print(df_raw.columns.tolist())

Размерность данных: (368, 47)

Колонки в данных:
['id', 'premium', 'name', 'department', 'has_test', 'response_letter_required', 'area', 'salary', 'salary_range', 'type', 'address', 'response_url', 'sort_point_distance', 'published_at', 'created_at', 'archived', 'apply_alternate_url', 'show_logo_in_search', 'show_contacts', 'insider_interview', 'url', 'alternate_url', 'relations', 'employer', 'snippet', 'contacts', 'schedule', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'fly_in_fly_out_duration', 'work_format', 'working_hours', 'work_schedule_by_days', 'night_shifts', 'professional_roles', 'accept_incomplete_resumes', 'experience', 'employment', 'employment_form', 'internship', 'adv_response_url', 'is_adv_vacancy', 'adv_context', 'branding', 'brand_snippet']


### 7. Сохраняем промежуточный CSV для анализа

In [15]:
# Сохраняем основные колонки
if not df_raw.empty:
    # Выбираем ключевые колонки
    columns_to_keep = [
        'id', 'name', 'area', 'salary', 'type', 'published_at',
        'created_at', 'archived', 'employer', 'snippet', 'experience',
        'employment', 'alternate_url'
    ]
    
    # Создаем DataFrame только с нужными колонками
    df_interim = df_raw[columns_to_keep].copy()
    
    # Сохраняем
    interim_path = f'../data/interim/vacancies_interim_{timestamp}.csv'
    df_interim.to_csv(interim_path, index=False, encoding='utf-8-sig')
    
    print(f"\nПромежуточные данные сохранены в: {interim_path}")
    print(f"Пример данных:")
    display(df_interim.head(3))
    
    # Базовая информация
    print("\nТипы данных:")
    print(df_interim.dtypes)
    
    print("\nПервые 5 названий вакансий:")
    for name in df_interim['name'].head():
        print(f"  - {name}")
else:
    print("Нет данных для сохранения")


Промежуточные данные сохранены в: ../data/interim/vacancies_interim_20251216_192026.csv
Пример данных:


,id,name,area,salary,type,published_at,created_at,archived,employer,snippet,experience,employment,alternate_url
0,128192065,Data Scientist / Data Analyst / Quantitative A...,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",2025-12-16T17:45:17+0300,2025-12-16T17:45:17+0300,False,"{'id': '5539742', 'name': 'Remokate', 'url': '...",{'requirement': 'Yandex School of <highlightte...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",https://hh.ru/vacancy/128192065
1,128061542,Senior Data Scientist – Renewable Energy Trading,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 5000, 'to': 7000, 'currency': 'USD', ...","{'id': 'open', 'name': 'Открытая'}",2025-12-16T17:19:56+0300,2025-12-16T17:19:56+0300,False,"{'id': '9544225', 'name': 'Trading Integral So...",{'requirement': 'Степень магистра или PhD в об...,"{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",https://hh.ru/vacancy/128061542
2,128096794,Data Scientist (ML Платформа),"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",2025-12-16T10:28:41+0300,2025-12-16T10:28:41+0300,False,"{'id': '4716984', 'name': 'X5 Digital', 'url':...",{'requirement': 'Опыт составления сложных SQL ...,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",https://hh.ru/vacancy/128096794



Типы данных:
id               object
name             object
area             object
salary           object
type             object
published_at     object
created_at       object
archived           bool
employer         object
snippet          object
experience       object
employment       object
alternate_url    object
dtype: object

Первые 5 названий вакансий:
  - Data Scientist / Data Analyst / Quantitative Analyst
  - Senior Data Scientist – Renewable Energy Trading
  - Data Scientist (ML Платформа)
  - Data Scientist в команду Интегрированного бизнес-планирования
  - Data Scientist, Anti-Fraud, Ozon Bank
